# Exercice de data crunching
 

## Installation des librairies nécessaires

- pandas
- datetime


**Documentation:**
- Tutoriel utilisé: https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/

In [225]:
import sys

# Installing pandas
!{sys.executable} -m pip install pandas

# Installing datetime
!{sys.executable} -m pip install datetime

# Installing numpy
!{sys.executable} -m pip install numpy

# Installing geopy
!{sys.executable} -m pip install geopy

## Importation des datasets



In [231]:
import pandas as pd
import datetime
import numpy as np
from geopy.distance import geodesic

# Importing CSVs
tickets_df = pd.read_csv('./csv/ticket_data.csv', comment='#')
cities_df = pd.read_csv('./csv/cities.csv', comment='#')
providers_df = pd.read_csv('./csv/providers.csv', comment='#')
stations_df = pd.read_csv('./csv/stations.csv', comment='#')

# PARTIE 1 : Stats d'un trajet choisi par l'utilisateur

## A. Sélection d'un trajet

**Documentation:**
- Doc pandas: https://pandas.pydata.org/pandas-docs/stable/user_guide/  
- Exemple supplémentaire StackOverflow: https://stackoverflow.com/questions/17071871/how-to-select-rows-from-a-dataframe-based-on-column-values

### Saisie des villes de départ et d'arrivée

In [64]:
# Get the trip origin and destination input
def getTripEndpoints():
    
    destination_city = input("You want to go to : ")
    origin_city = input("from : ")
    
    return getCityId(origin_city), getCityId(destination_city);

### Récupération de l'id d'une ville

In [100]:
# Get the id of a given city
def getCityId(city):
    
    # The cities are in lowercase in the column we're searching
    city = city.lower()
    
    # If the city isn't in the dataframe
    if(cities_df.loc[cities_df['unique_name'].isin([city])].empty):
        
        # Not very clean exit
        print('We couln\'t find this city')
        sys.exit(0)
    
    # Find the correct row in the cities dataframe
    else:
        
        # Get the id
        cityId = cities_df.loc[cities_df['unique_name'] == city, 'id'].item()
    
    return cityId;

### Sélection de tous les tickets pour un trajet

In [101]:
# Function returning a dataframe with all the trips between these cities
def getTrips(tickets_df, originCity = 628, destinationCity = 453):
    
    # Selecting the trips between origin and destination
    selected_trips_df = tickets_df.loc[(tickets_df['o_city'] == originCity) & (tickets_df['d_city'] == destinationCity)]
    
    return selected_trips_df;

## B. Prix des tickets

In [67]:
# Getting the minimum and the maximum price of the selected trip and the mean of the prices

def getPricesBounds(trips_df):
    minPrice = (trips_df['price_in_cents'].min()) / 100
    avgPrice = (trips_df['price_in_cents'].mean()) / 100
    maxPrice = (trips_df['price_in_cents'].max()) / 100
    
    priceBounds = pd.Series([minPrice, avgPrice, maxPrice], index=list(['minimum', 'average', 'maximum']))
    
    return priceBounds;

## C. Durées des trajets

### Ajout d'une colonne 'durée du trajet'

**Documentation:**
- https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html
- https://pandas.pydata.org/pandas-docs/stable/user_guide/timedeltas.html

**Problème rencontré:**  
- *SettingWithCopyWarning*

    Solutions trouvées: 
    - https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
    - https://stackoverflow.com/questions/36846060/how-to-replace-an-entire-column-on-pandas-dataframe

In [68]:
# Adds a duration column and format the timestamps to datetime
def addDuration(trips_df):
    
    # Changing departure and arrival to datetime
    formatted_trips_df = trips_df.loc[:, ['departure_ts', 'arrival_ts']].apply(pd.to_datetime)
    
    # Adding all the other columns
    formatted_trips_df = trips_df.assign(departure_ts = formatted_trips_df['departure_ts'], arrival_ts = formatted_trips_df['arrival_ts'])
    
    
    # Create a Serie containing the durations between departure and arrival
    duration_s = formatted_trips_df.loc[:, 'arrival_ts'] - formatted_trips_df.loc[:, 'departure_ts']
    duration_s.name = 'duration_tdelt'
    
    # Adding the durations to the dataframe
    formatted_trips_df = pd.concat([formatted_trips_df, duration_s], axis = 1)
    
    return formatted_trips_df;

### Calcul des durées min, max et moyennes

**Documentation:**  
- Convertir un timedelta en secondes: https://pandas.pydata.org/pandas-docs/stable/user_guide/timedeltas.html#conversions

In [69]:
def getDurationBounds(trips_df):
    
    # Min and max duration
    minDuration = min(trips_df.loc[:, 'duration_tdelt'])
    maxDuration = max(trips_df.loc[:, 'duration_tdelt'])
    
    # Converting the durations in seconds
    durationInSecs = trips_df.loc[:, 'duration_tdelt'].astype('timedelta64[s]')
    
    # Computing the mean and returning it in timedelta
    avgDuration = durationInSecs.mean()
    avgDuration = pd.to_timedelta(avgDuration, 'seconds')
    
    durationBounds_s = pd.Series([minDuration, avgDuration, maxDuration], index=list(['minimum', 'average', 'maximum']))
    
    return durationBounds_s;

## D. Affichage

### Affichage d'un timestamp au format HH:mm   

**Documentation:**  
- Formater un timedelta: https://stackoverflow.com/questions/538666/format-timedelta-to-string

In [70]:
# Display of a timedelta
def timedeltaDisplay(timedelta):
    
    # Get the total # of seconds
    timedeltaSeconds = timedelta.seconds
    
    # Translate it in hours and minutes
    hours, remainder = divmod(timedeltaSeconds,3600)
    minutes = remainder // 60
    
    # Build the string
    display = '{:02d}h{:02d}mn'.format(hours, minutes)
    return display;

### Récupération du nom complet d'une ville

In [71]:
# Get the city, its region and its country 
def getCityLocalName(city_id):
    
    # Search the city local name using id
    cityLocalName = cities_df.loc[cities_df['id'] == city_id, 'local_name'].item()
    
    # Split the city, the region and the country    
    cityLocation = pd.Series(cityLocalName.split(', '), index=list(['city', 'region', 'country']))
    
    return cityLocation;

### Affichage des résultats

**Documentation:**  
- Affichage des durées moyennes: https://stackoverflow.com/questions/538666/format-timedelta-to-string

In [72]:
# Display the stats of the trips available
def displayTripStats(origin_city_id, destination_city_id, priceBounds, durationBounds):
    
    # Get the cities full names
    origin = getCityLocalName(origin_city_id)
    destination = getCityLocalName(destination_city_id)
    
    # Print trip endpoints
    print("\n")
    print("Trip from {} ({}) to {} ({}):\n"
          .format(origin.city, origin.region, destination.city, destination.region))
         
    # Print prices available
    print("Prices : {:.2f}€ >>> {:.2f}€. Average: {:.2f}€"
          .format(priceBounds.minimum, priceBounds.maximum, priceBounds.average))
    
    # Print trip durations
    print("Duration : {} >>> {}. Average: {}"
          .format(
              timedeltaDisplay(durationBounds.minimum), 
              timedeltaDisplay(durationBounds.maximum), 
              timedeltaDisplay(durationBounds.average)))
    
    return;

## E. Main

Reste à gérer le cas où il n'y a aucun trajet entre les deux villes sélectionnées

In [202]:
# Get the ids of the destination and the origin city entered 
originCityId, destinationCityId = getTripEndpoints()

# Select the trips available between the given cities
myTrip_df = getTrips(tickets_df, originCityId, destinationCityId)

# Get price overview
priceBounds_s = getPricesBounds(myTrip_df)

# Add a duration column for each trip
myTrip_df = addDuration(myTrip_df)

# Get duration overview
durationBounds_s = getDurationBounds(myTrip_df)


# Display the results
displayTripStats(originCityId, destinationCityId, priceBounds_s, durationBounds_s)

You want to go to : london
from : paris


Trip from Paris (Île-de-France) to London (England):

Prices : 35.00€ >>> 263.40€. Average: 89.34€
Duration : 03h59mn >>> 11h32mn. Average: 09h43mn


# PARTIE 2 : Prix selon la distance et le moyen de transport

## A. Sélection de trajets selon certains critères

### Récupérer les trajets utilisant le même moyen de transport

**Problème rencontré:**

Je ne sais pas lire, donc je perds du temps

**Solution:**

Relire

In [207]:
def getByTransportType(trips_df, transportType):

    # Get all the ids of providers matching the selected transport type
    companyByTransportType = providers_df.loc[providers_df['transport_type'] == transportType, 'id']

    # Select the trips from the selected providers
    selectedTrips_df = trips_df.loc[trips_df['company'].isin(companyByTransportType)]
    
    return selectedTrips_df;

### Découper un dataframe selon les moyens de transports

In [208]:
# Returns 3 separated dataframes for train, bus and carpooling trip tickets
def separateTransportTypes(trips_df):
    
    trainTrips_df = getByTransportType(trips_df, 'train')
    busTrips_df = getByTransportType(trips_df, 'bus')
    carpoolingTrips_df = getByTransportType(trips_df, 'carpooling')
    
    return trainTrips_df, busTrips_df, carpoolingTrips_df;

### Comparer les prix selon le moyen de transport

In [213]:
# Returns the average price for each mean of transport
def comparePrices(trainTrip_df, busTrip_df, carpoolingTrip_df):
    
    # Computes the average
    avgTrainPrice = trainTrip_df.price_in_cents.mean() / 100
    avgBusPrice = busTrip_df.price_in_cents.mean() / 100
    avgCarpoolingPrice = carpoolingTrip_df.price_in_cents.mean() / 100
    
    # Gather the average for each way of transportation
    avgPrices = pd.Series([avgTrainPrice, avgBusPrice, avgCarpoolingPrice], index=list(['train', 'bus', 'carpooling']))
    
    return avgPrices;

### Calcul de la distance d'un trajet

**Documentation:**

- geopy: https://pypi.org/project/geopy/

**Démarche:**

- Récupérer toutes les villes de départ/arrivée + stations de départ/arrivée + stations intermédiaires
- Supprimer les doublons
- Récupérer les coordonnées de toutes les villes / stations
    - Ajouter o_station au début, sinon ajouter o_city
    - Ajouter middle_stations, si définies
    - Ajouter d_station à la fin, sinon ajouter d_city
- Calculer les distances associées
- Ajouter la colonne distance à tous les trajets


In [256]:
distances_df = tickets_df.loc[:,['o_station', 'd_station', 'middle_stations', 'o_city', 'd_city']]
distances_df.drop_duplicates()


distances_df

,o_station,d_station,middle_stations,o_city,d_city
0,NaN,NaN,NaN,611,542
1,63.0,1044.0,"{149,418}",611,542
2,5905.0,6495.0,"{798,798,6794,6246}",611,542
3,5905.0,6495.0,"{798,798,6794,6246}",611,542
4,5905.0,6495.0,"{5983,5983}",611,542
...,...,...,...,...,...
74163,279.0,10729.0,"{408,408}",628,562
74164,279.0,304.0,"{1105,1105}",628,562
74165,10642.0,304.0,"{1105,1105}",628,562
74166,279.0,304.0,"{863,863}",628,562


## Main

In [214]:
# Add a duration column for every row of the processed dataset
myTrip_df = addDuration(tickets_df)

myTrainTrip_df, myBusTrip_df, myCarpoolingTrip_df = separateTransportTypes(myTrip_df)

pricesRange = comparePrices(myTrainTrip_df, myBusTrip_df, myCarpoolingTrip_df)

print(pricesRange)

train         85.066348
bus           36.524480
carpooling    27.421719
dtype: float64


In [242]:
stations_df.loc[stations_df['id'] == 304]

,id,unique_name,latitude,longitude
303,304,Gare de Haluchère-Batignolles,47.248785,-1.520898
